In [1]:
import sys
reload(sys)  # need to reload to set default encoding
sys.setdefaultencoding('utf8')

import argparse
import collections
import json
import logging
import numpy as np
import os
import shutil
import tensorflow as tf
import pandas as pd
import itertools
import re
import pickle
import random

In [0]:
import matplotlib
matplotlib.rc('font', family='DejaVu Sans')
from matplotlib import pyplot


In [0]:
# logging
# output_vocab.Save(output_vocab_filename)
logging.basicConfig(filename='logfile.log',filemode='w', level=logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [0]:
config = tf.ConfigProto(inter_op_parallelism_threads=10, intra_op_parallelism_threads=10)
tf.set_random_seed(666)

In [0]:
f = open('./train.txt', 'r')
f1 = f.readlines()
sentences = []
words = []
labels = []
i = 0
line = f1[0]
for line in f1:
# for i in xrange(2):
#   line = f1[i]
  sentences.append(line)
  w1 = line.split()
  word = [] 
  label = []
  for w in w1:
    w2 = w.split('/')
#     if(w2[1]=='amb' or w2[1]=='ne'):
#       continue
    
  
    word.append(w2[0])   
    label.append(w2[1])    
  words.append(word)  
  labels.append(label)  

In [0]:
# line
# labels[1]
# # words[1]


In [0]:
train_IDs = []
eval_IDs = []
train_Lang = []
eval_Lang = []
train_Words_unicode = []
eval_Words_unicode = []
tweetID = 0
for row in words:
  if tweetID < 0.8 * len(words):
    train_IDs.append(tweetID)
    train_Lang.append(labels[tweetID])
    #train_Words_unicode.append([unicode(x_.decode('utf8')) for x_ in row])
    train_Words_unicode.append(['{'] + [unicode(x_.decode('utf8')) for x_ in row] + ['}'])
    
  else:
    eval_IDs.append(tweetID)
    eval_Lang.append(labels[tweetID])
#     eval_Words_unicode.append([unicode(x_.decode('utf8')) for x_ in row])
    eval_Words_unicode.append(['{'] + [unicode(x_.decode('utf8')) for x_ in row] + ['}'])
  tweetID = tweetID + 1

In [0]:
# batch_size = 1
batch_size = 25
und_symbol = 'und'
weight = 1.0
maxlen = 40

dataset_weights = []
dataset_weights.append(weight)

################## vocab ##################################

class Vocab(object):

  def _normalize(self, word):
    if re.match(ur'^<.*>$', word):
      return word

    if self.specialcase:
      newword = []
      prev_is_lower = True
      for letter in word:
        if letter.isupper():
          if prev_is_lower:
            newword.append('~')
          prev_is_lower = False
        else:
          prev_is_lower = True
        newword.append(letter.lower())
      word = ''.join(newword)

    if self.lowercase:
      word = word.lower()

    if self.numreplace:
      word = re.sub(ur'\d', '#', word)

    return word

  def __init__(self, tokenset, unk_symbol='<UNK>',
               lowercase=False, numreplace=False):
    self.lowercase = lowercase
    self.numreplace = numreplace

    self.specialcase = False
    
    
    tokenset = set([self._normalize(w) for w in tokenset])   #########

    self.vocab_size = len(tokenset)
    self.unk_symbol = unk_symbol

    self.word_to_idx = dict(zip((tokenset), range(self.vocab_size)))
    self.idx_to_word = dict(zip(self.word_to_idx.values(),
                                self.word_to_idx.keys()))
    self.t = tokenset
  @staticmethod
  def Load(filename):
    with open(filename, 'rb') as f:
      v = pickle.load(f)
    return v

  @classmethod
  def MakeFromData(cls, lines, min_count, unk_symbol='<UNK>',
                   max_length=None, no_special_syms=False, normalize=False):
    lowercase=False
    numreplace=False

    if normalize:
      lowercase=True
      numreplace=True

    token_counts = collections.Counter()

    for line in lines:
      token_counts.update(line)

    tokenset = set()
    for word in token_counts:
      if max_length and len(word) > max_length:      #############
        continue
      if token_counts[word] >= min_count:
        tokenset.add(word)

    if not no_special_syms:
      tokenset.add(unk_symbol)
      tokenset.add('<S>')
      tokenset.add('</S>')

    return cls(tokenset, unk_symbol=unk_symbol, lowercase=lowercase,
               numreplace=numreplace)

  @classmethod
  def ByteVocab(cls):
    """Creates a vocab that has a token for each possible byte.

    It's useful to have a fixed byte vocab so that the subset of bytes
    that form the vocab is not dependent on the dataset being used. Thus,
    the learned byte embeddings can be reused on different datasets.
    """
    c = '0123456789abcdef'
    tokens = ['<S>', '</S>']
    for i in c:
      for j in c:
        tokens.append(i + j)
    return cls(tokens)

  @classmethod
  def LoadFromTextFile(cls, filename, unk_symbol='<UNK>'):
    tokens = []
    with open(filename, 'r') as f:
      for line in f:
        line = line.strip()
        tokens.append(line)
    return cls(tokens, unk_symbol=unk_symbol)

  def GetWords(self):
    """Get a list of words in the vocabulary."""
    return self.word_to_idx.keys()

  def LookupIdx(self, token):
    token = self._normalize(token)
    if token in self.word_to_idx:
      return self.word_to_idx[token]
    return self.word_to_idx.get(self.unk_symbol, None)

  def __contains__(self, key):
    key = self._normalize(key)
    return key in self.word_to_idx

  def __getitem__(self, key):
    """If key is an int lookup word by id, if key is a word then lookup id."""
    if type(key) == int or type(key) == np.int64:
      return self.idx_to_word[key]

    return self.LookupIdx(key)

  def __iter__(self):
    word_list = [self.idx_to_word[x] for x in xrange(self.vocab_size)]
    return word_list.__iter__()

  def __len__(self):
    return self.vocab_size

  def Save(self, filename):
    if filename.endswith('.pickle'):
     with open(filename, 'wb') as f:
      pickle.dump(self, f)
    elif filename.endswith('.txt'):
      with open(filename, 'w') as f:
        for i in range(self.vocab_size):
          f.write('{0}\n'.format(self.idx_to_word[i]))


    
###########################################################################


input_vocab = Vocab.MakeFromData(train_Words_unicode, max_length=maxlen, min_count=1)
test_input_vocab = Vocab.MakeFromData(eval_Words_unicode, max_length=maxlen, min_count=1)
# input_vocab = Vocab.MakeFromData(itertools.chain(train_Words_unicode), max_length=maxlen, min_count=1)

  
def Graphemes(s):
  graphemes = []
  current = []

  if type(s) == unicode:
    s = s.encode('utf8')

  for c in s:
    val = ord(c) & 0xC0
    if val == 128:
      # this is a continuation
      current.append(c)
    else:
      # this is a new grapheme
      if len(current) > 0:
        graphemes.append(''.join(current))
        current = []

      if val < 128:
        graphemes.append(c)  # single byte grapheme
      else:
        current.append(c)  # multi-byte grapheme

  if len(current) > 0:
    graphemes.append(''.join(current))

  return graphemes

  
###################  training ####################

# x = [util.Graphemes(w) for w in input_vocab.GetWords()]
x = [Graphemes(w) for w in input_vocab.GetWords()]      #########
char_vocab = Vocab.MakeFromData(x, min_count=2)         #######
char_vocab.Save('./sample_data/char_vocab.pickle')      ########

train_label_set = set()
for d in train_Lang:
  train_label_set.update(d)

eval_label_set = set()
for d in eval_Lang:
  eval_label_set.update(d)

output_vocab_filename = './sample_data/out_vocab.pickle'
labels = [[x1] for x1 in train_label_set]
if [und_symbol] not in labels: labels += [[und_symbol]]
output_vocab = Vocab.MakeFromData(labels, min_count=1, no_special_syms=True)
output_vocab.Save(output_vocab_filename)

test_output_vocab_filename = './sample_data/test_out_vocab.pickle'
eval_labels = [[x1] for x1 in eval_label_set]
if [und_symbol] not in eval_labels: eval_labels += [[und_symbol]]
test_output_vocab = Vocab.MakeFromData(eval_labels, min_count=1, no_special_syms=True)
test_output_vocab.Save(test_output_vocab_filename)


# with open(args.params, 'r') as f:
#   model_params = json.load(f)

# with open(os.path.join(args.expdir, 'model_params.json'), 'w') as f:
#   json.dump(model_params, f)


In [0]:
input_vocab.vocab_size

train_label_set
labels

[['oth'], ['en'], ['bn'], ['amb'], ['ne'], ['as'], ['und']]

In [0]:
######################## Dataset ################################
counts=0
class Dataset(object):

  def __init__(self, batch_size, preshuffle=False, name='unnamed'):
    """Init the dataset object.

    Args:
      batch_size: size of mini-batch
      preshuffle: should the order be scrambled before the first epoch
      name: optional name for the dataset
    """
    self._sentences = []
    self._labels = []
    self._ids = []
    self.dataset_weights = []
    self._lines = []
    self.name = name

    self.batch_size = batch_size
    self.preshuffle = preshuffle

  def ReadData(self, filename, mode, modeltype, weight=1.0):
    d = LoadData(filename, mode, modeltype)
    self.AddDataSource(d, weight=weight)

  def AddDataSource(self, data, weight=1.0):
    sentences, labels, ids = data
    self._sentences.append(sentences)
    self._labels.append(labels)
    self.dataset_weights.append(weight)
    self._ids.append(ids)

  def GetSentences(self):
    return itertools.chain(*self._sentences)

  def GetIds(self):
    return [x for x in itertools.chain(*self._ids)]

  def GetLabelSet(self):
    label_set = set()
    for d in self._labels:
      label_set.update([x for x in itertools.chain(*d)])
    return label_set

  def Prepare(self,in_vocab,out_vocab,und_label='und',ignore_categories=[]):
    # Add a dummy dataset to make the batch size evenly divide the number
    # of sentences.
    batch_size = self.batch_size
    total_sentences = sum([len(x) for x in self._sentences])
    r = total_sentences % batch_size
    if r > 0:
      n = batch_size - r
      self.AddDataSource(([list('{dummy}')] * n, [[und_label]] * n, [0] * n),
                         weight=0.0)

    sentences = list(itertools.chain(*self._sentences))
    labels = list(itertools.chain(*self._labels))

#     self.example_weights = []
#     for i in xrange(len(self.dataset_weights)):
#       w = self.dataset_weights[i]
#       for j in xrange(len(self._sentences[i])):
#         ex_wt = []
#         for _ in xrange(len(self._sentences[i][j])):
#           ex_wt.append(w)
#         self.example_weights.append(ex_wt)
#     self.example_weights = np.array(self.example_weights)

    self.example_weights = []
    for i in xrange(len(self.dataset_weights)):
      w = self.dataset_weights[i]
      for _ in xrange(len(self._sentences[i])):
        self.example_weights.append(w)
    self.example_weights = np.array(self.example_weights)


    self.seq_lens = np.array([len(x) for x in sentences])
#     self.max_sequence_len = self.seq_lens.max()
    self.max_sequence_len = 0
    for x in self.seq_lens:
      if x > self.max_sequence_len:
        self.max_sequence_len = x

    self.batch_size = batch_size
    self.current_idx = 0
    self.max_sequence_len = 2453               #############################
    self.sentences = self.GetNumberLines(sentences, in_vocab,
                                         self.max_sequence_len)
    self.labels = np.zeros((len(labels), self.max_sequence_len, len(out_vocab)))
#     label_size = 4
    label_size = 7
    for i, w in enumerate(labels):
      self.labels[i,0,label_size-1] = 1.0
      z = 1
      for w_ in w:
        print w_ +" "+ str(i) +" "+str(z)+" "+str(out_vocab[w_])
        self.labels[i,z, out_vocab[w_]] = 1.0
        z=z+1
      while (z<2453):
        self.labels[i,z,label_size-1] = 1.0
        z=z+1
#       self.labels[i, :] /= self.labels[i, :].sum()
#     for i, w in enumerate(labels):
#       for w_ in w:
#         self.labels[i, out_vocab[w_]] = 1.0
#       self.labels[i, :] /= self.labels[i, :].sum()
    # class weights
    # There is a hack to only use the examples with weight 1 as a way
    # to prevent wikipedia from dominating the weights.
    
#     counts = self.labels[np.sum(self.example_weights, axis = 1) == 1, :].sum(axis=0)
    counts = self.labels[self.example_weights == 1, :].sum(axis=0)
    self.w = 1.0/(1.0 + counts)
    self.w /= self.w.mean()  # scale the class weights to reasonable values
#     self.w = 1.0
    
    self.N = len(sentences)
    if self.preshuffle:
      self._Permute()

  @staticmethod
  def GetNumberLines(lines, vocab, pad_length):
    """Convert list of words to matrix of word ids."""
    out = []
    for line in lines:
      if len(line) < pad_length:
        line += ['}'] * (pad_length - len(line))       #########
      out.append([vocab[w] for w in line])            #########
    return np.array(out)                            #######

  def GetNumBatches(self):
    """Returns num batches per epoch."""
    return self.N / self.batch_size

  def _Permute(self):
    """Shuffle the training data."""
    s = np.arange(self.N)
    np.random.shuffle(s)

    self.sentences = self.sentences[s, :]
    self.seq_lens = self.seq_lens[s]
    self.labels = self.labels[s, :]
    self.example_weights = self.example_weights[s]

  def GetNextBatch(self):
    if self.current_idx + self.batch_size > self.N:
      self.current_idx = 0
#       self._Permute()

    idx = range(self.current_idx, self.current_idx + self.batch_size)
    self.current_idx += self.batch_size

    return (self.sentences[idx, :], self.seq_lens[idx],
            self.labels[idx, :], self.example_weights[idx])

    #########################################################################

In [0]:
dataset = Dataset(batch_size)
test_dataset = Dataset(batch_size)

train_data = [train_Words_unicode, train_Lang, train_IDs]
test_data = [eval_Words_unicode, eval_Lang, eval_IDs]

dataset.AddDataSource(train_data)
dataset.Prepare(input_vocab, output_vocab, und_symbol)

test_dataset.AddDataSource(test_data)
test_dataset.Prepare(test_input_vocab, test_output_vocab, und_symbol)

In [0]:
# x
# char_vocab.word_to_idx

In [0]:
class Char2Vec(object):
  """Maps character sequences to word embeddings."""

  def __init__(self, char_vocab, max_sequence_len=15, initializer = 0):
    """Initialize the Char2Vec model.

    Args:
      char_vocab: vocab class instance for the character vocabulary
      max_sequence_len: length of the longest word
    """
    self.max_sequence_len = max_sequence_len
    self.char_vocab = char_vocab
    self._vocab_size = char_vocab.vocab_size

    # Placeholder for the character sequences in the form of an
    # n x k array where n is the number of words and k is the
    # length of the longest word. Characters are encoded as ints.
    self.words_as_chars = tf.placeholder(tf.int32, [None, max_sequence_len],
                                         name='words_as_chars')
#     self.words_as_chars = tf.Variable(tf.zeros([None, max_sequence_len]), name='words_as_chars', dtype=tf.int32, initializer=initializer)
#     self.words_as_chars = tf.get_variable("words_as_chars",
#                                      [None, max_sequence_len],
#                                      initializer=initializer, dtype=tf.int32)

  def GetEmbeddings(self, x):
    return tf.nn.embedding_lookup(self.word_embeddings, x)

  def MakeMat(self, word_list, pad_len=None):
    """Make a matrix to hold the character sequences in.

    Special start and end tokens are added to the beggining and end of
    each word.

    Args:
      word_list: A list of strings
      pad_len: Pad all character sequences to this length. If a word is
               longer than the pad_len it will be truncated.

    Returns:
      Array containing character sequences and a vector of sequence lengths.
    """
    if not pad_len:
      pad_len = self.max_sequence_len

    # make the padded char mat
    the_words = []
    word_lengths = []
    for word in word_list:
      word_idx = [self.char_vocab[c] for c in Graphemes(word)]
      word_idx = ([self.char_vocab['<S>']] + word_idx[:pad_len-2] +
                  [self.char_vocab['</S>']])
      if len(word_idx) < pad_len:
        word_idx += [self.char_vocab['</S>']] * (pad_len - len(word_idx))
      the_words.append(word_idx)
      word_lengths.append(min(pad_len, len(word)+2))

    the_words = np.array(the_words)
    word_lengths = np.array(word_lengths)
    return the_words, word_lengths

  @staticmethod
  def GetBatchVocab(words):
    batch_vocab = np.unique(words)
    words_remapped = np.copy(words)
    for i in xrange(len(batch_vocab)):
      np.place(words_remapped, words==batch_vocab[i], i)
    return batch_vocab, words_remapped

class CharCNN(Char2Vec):
  """CNN implementation of char2vec.

  The model uses two layers of convolution. The second one is followed by a
  max pooling operation. After that there is a resnet layer.
  """

  def __init__(self, char_vocab,
               max_sequence_len=15, dropout_keep_prob=None, initializer = 0):
    super(CharCNN, self).__init__(char_vocab, max_sequence_len, initializer = initializer)

    char_embed_dims = int(np.log(len(char_vocab))) + 1

    layer1_out_size = 60
    hidden_size = 70
    word_embed_dims = 80

    # The following variables define the model.
    with tf.variable_scope('c2v', reuse=tf.AUTO_REUSE):
      
      # continuous space character embeddings
      self.embedding = tf.get_variable("embedding",
                                       [self._vocab_size, char_embed_dims],
                                       initializer=initializer)
      the_filter, filter_b = MakeFilter(3, char_embed_dims, layer1_out_size,
                                        'filt')

      # z is a tensor of dimensions batch_sz x word_len x embed_dims.
      z = tf.nn.embedding_lookup(self.embedding, self.words_as_chars)
      z_expanded = tf.expand_dims(z, -1)

      conv = tf.nn.conv2d(z_expanded, the_filter, strides=[1, 1, 1, 1],
                          padding='VALID' )
      h = tf.nn.relu(tf.nn.bias_add(tf.squeeze(conv), filter_b))
      h.set_shape((None, max_sequence_len - 2, layer1_out_size))
      if dropout_keep_prob is not None:
        h_expanded = tf.nn.dropout(tf.expand_dims(h, -1), dropout_keep_prob)
      else:
        h_expanded = tf.expand_dims(h, -1)

      pools = []
      filter_sizes = range(3,6)
      for width in filter_sizes:
        f, f_bias = MakeFilter(width, layer1_out_size, hidden_size,
                               'filter_w{0}'.format(width))
        conv2 = tf.nn.conv2d(h_expanded, f, strides=[1, 1, 1, 1],
                             padding='VALID')
        h2 = tf.nn.relu(tf.nn.bias_add(conv2, f_bias))
        pooled = tf.nn.max_pool(h2, ksize=[1, max_sequence_len-1-width, 1, 1],
                                strides=[1, 1, 1, 1], padding='VALID')
        pools.append(pooled)

        if width == 3:  # debugging
          self.hh = tf.squeeze(pooled)
          self.hidx = tf.argmax(h2, 1)

      pooled = tf.squeeze(tf.concat(pools,3), [])

      # resnet layer https://arxiv.org/abs/1512.03385
      sz = len(filter_sizes) * hidden_size
      t_mat = tf.get_variable('t_mat', [sz, sz])
      t_bias = tf.Variable(tf.constant(0.1, shape=[sz]), name='t_bias')
      t = tf.nn.relu(tf.matmul(pooled, t_mat) + t_bias)

 
      self.word_embeddings = t + pooled
      self.embedding_dims = sz

#       self.word_embeddings = tf.placeholder(tf.float32, [None, max_sequence_len],
#                                          name='word_embeddings')

def MakeFilter(width, in_size, num_filters, name):
  filter_sz = [width, in_size, 1, num_filters]
  filter_b = tf.Variable(tf.constant(0.1, shape=[num_filters]),
                         name='{0}_bias'.format(name))
  the_filter = tf.get_variable(name, filter_sz)
  return the_filter, filter_b


def reverse_seq(input_seq, lengths):
  """Reverse a list of Tensors up to specified lengths.
  Args:
    input_seq: Sequence of seq_len tensors of dimension (batch_size, depth)
    lengths:   A tensor of dimension batch_size, containing lengths for each
               sequence in the batch. If "None" is specified, simply reverses
               the list.
  Returns:
    time-reversed sequence
  """
  for input_ in input_seq:
    input_.set_shape(input_.get_shape().with_rank(2))

  # Join into (time, batch_size, depth)
  s_joined = tf.pack(input_seq)

  # Reverse along dimension 0
  s_reversed = tf.reverse_sequence(s_joined, lengths, 0, 1)
  # Split again into list
  result = tf.unpack(s_reversed)
  return result


In [0]:
initializer = tf.random_uniform_initializer(-0.1, 0.1)

max_word_len = max([len(x) for x in input_vocab.GetWords()]) + 2
# with tf.variable_scope(tf.get_variable_scope(), reuse=True):
c2v = CharCNN(char_vocab,max_sequence_len=max_word_len,dropout_keep_prob=0.7, initializer = initializer)
the_words, word_lengths = c2v.MakeMat(input_vocab, pad_len=max_word_len)

In [0]:
c2v.embedding_dims

210

In [0]:
# c2v.word_embeddings.shape
c2v.words_as_chars

<tf.Tensor 'words_as_chars:0' shape=(?, 24) dtype=int32>

In [0]:
class WordLevelModel(object):
  """
    Model to evaluate on word-level predictions

    Args:
      batch_size: minibatch size
      model_params: dictionary of other model parameters
      c2v: char2vec class instance
      max_sequence_len: length of all the input/output sequences
      out_vocab_size: how many languages we are predicting
      dropout_keep_prob: dropout probability indicator
      weights: class weights
  """

  def __init__(self, batch_size, c2v, max_sequence_len,
               out_vocab_size, dropout_keep_prob=0.0, weights=None):
    self._batch_size = batch_size
    self._dropout_keep_prob = dropout_keep_prob
    self._out_vocab_size = out_vocab_size

    self.x = tf.placeholder(tf.int32, [batch_size, max_sequence_len],
                            name='x')
    self.y = tf.placeholder(tf.float32,
                            [batch_size, max_sequence_len, out_vocab_size],
                            name='y')
    # The bidirectional rnn code requires seq_lens as int64
    self.seq_lens = tf.placeholder(tf.int64, [batch_size], name='seq_lens')
#     self.example_weights = tf.placeholder(tf.float32, [batch_size, max_sequence_len],
#                                          name='example_weights')
#     self.example_weights = tf.placeholder(tf.float32, [batch_size],
#                                          name='example_weights')

    embeddings = c2v.GetEmbeddings(self.x)        ##before
    self.temp = embeddings
#     embeddings = tf.reshape(embeddings, [batch_size, max_sequence_len, 50])    ###########
    embeddings = tf.reshape(embeddings, [batch_size, max_sequence_len, 210])
#     tf.squeeze(embeddings)
    self._inputs = embeddings
  
    # Need to prepare a mask to zero out the padding symbols.

    # Make a batch_size x max_sequence_len matrix where each
    # row contains the length repeated max_sequence_len times.
    lengths_transposed = tf.expand_dims(self.seq_lens, 1)
#     lengths_transposed = tf.expand_dims(tf.to_int64(self.seq_lens), 1)
    lengths_tiled = tf.tile(lengths_transposed, [1, max_sequence_len])

    # Make a matrix where each row contains [0, 1, ..., max_sequence_len]
    r = tf.range(0, max_sequence_len, 1)
    range_row = tf.expand_dims(r, 0)
    range_tiled = tf.tile(range_row, [batch_size, 1])

    self.lengths_transposed = lengths_transposed
    self.lengths_tiled = lengths_tiled
    self.range_row = range_row
    self.range_tiled = range_tiled

    # Use the logical operations to create a mask
    indicator = tf.less(range_tiled, tf.to_int32(lengths_tiled+1)) #i.e. where seq len is less than index
    trim = np.ones(indicator.get_shape())
    trim[:,0] = 0 #ignore start symbol
    indicator = tf.logical_and(indicator, trim.astype(bool))
    self.indicator = indicator

    sz = [batch_size, max_sequence_len]
    self._mask = tf.where(indicator, tf.ones(sz), tf.zeros(sz))

    #-------------------------------#

    self.weights = tf.constant(weights, dtype=tf.float32, name='class_weights')

    hidden_size = 100
#     hidden_size = 40
    proj_size = None
    def GetCell():
      """Creates an LSTM cell with dropout."""
      c = tf.nn.rnn_cell.LSTMCell(hidden_size,
                                  use_peepholes=False,
                                  num_proj=proj_size)
      if dropout_keep_prob is not None:
        c = tf.nn.rnn_cell.DropoutWrapper(c, input_keep_prob=dropout_keep_prob)
      return c

    # Create the bi-directional LSTM
    with tf.variable_scope('wordrnn', reuse=tf.AUTO_REUSE):
      with tf.variable_scope('fw', reuse=tf.AUTO_REUSE):
        cell_fw = GetCell()
      with tf.variable_scope('bw', reuse=tf.AUTO_REUSE):
        cell_bw = GetCell()
        

      rnnout, _ = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, self._inputs,
                                             dtype=tf.float32,
                                             sequence_length=self.seq_lens)

      if proj_size:
        out_size = 1 * proj_size
#         out_size = 2 * proj_size
      else:
        out_size = 1 * hidden_size
#         out_size = 2 * hidden_size
      self._DoPredictions(out_size, rnnout, self.weights)

      self.cost = tf.reduce_mean(self._xent)
#       self.cost = tf.reduce_mean(tf.multiply(self._xent,tf.expand_dims(self._mask,2)))
#       self.cost = tf.reduce_mean(self.example_weights * self._xent)

  def _DoPredictions(self, in_size, mats, class_weights=None):
    """Takes in an array of states and calculates predictions.

    Get the cross-entropy for each example in the vector self._xent.

    Args:
      in_size: size of the hidden state vectors
      mats: list of hidden state vectors
    """
    pred_mat = tf.get_variable('pred_mat0',
                               [in_size, self._out_vocab_size])
    pred_bias = tf.get_variable('pred_bias', [self._out_vocab_size])

    # Make a prediction on every word.
    def GetWordPred(o_):
      logits = tf.nn.xw_plus_b(o_, pred_mat, pred_bias)
      return tf.nn.softmax(logits)

    #self.preds_by_word1 = tf.pack([GetWordPred(o_) for o_ in mats])
    #self.preds_by_word = tf.reshape(self.preds_by_word1, self.y.get_shape())
    #self.probs = tf.mul(tf.expand_dims(self._mask,2), self.preds_by_word)

#     self.preds_by_word = tf.pack([GetWordPred(o_) for o_ in mats])
#     self.preds_by_instance = tf.pack([self.preds_by_word[:,i,:] for i in range(self.preds_by_word.get_shape()[1])])
#     self.probs = tf.mul(tf.expand_dims(self._mask,2), self.preds_by_instance)

    self.preds_by_word = tf.stack([GetWordPred(o_) for o_ in mats])
    self.preds_by_instance = tf.stack([self.preds_by_word[:,i,:] for i in range(self.preds_by_word.get_shape()[1])])
    self.probs1 = tf.multiply(tf.expand_dims(self._mask,2), tf.reduce_sum(self.preds_by_instance,1))
    self.prob_div = tf.reduce_sum(self.probs1,2)
    self.probs = self.probs1/tf.expand_dims(self.prob_div,-1)
#     self.probs = tf.multiply(tf.expand_dims(tf.expand_dims(self._mask,1),3), tf.reduce_sum(self.preds_by_instance,2))
#     self.probs = tf.multiply(tf.expand_dims(self._mask,2), self.preds_by_word)
#     self.probs = tf.multiply(tf.expand_dims(self._mask,2), self.preds_by_instance)

#     self._xent = _SafeXEnt(self.y, self.probs, class_weights=class_weights, sumd=[2])
    self._xent, self.adjprob = _SafeXEnt(self.y, self.probs1, class_weights=class_weights, sumd=[1,2])


def _SafeXEnt(y, probs, eps=0.0001, class_weights=None, sumd=[1]):
  """Version of cross entropy loss that should not produce NaNs.

  If the predicted proability for the true class is near zero then when
  taking the log it can produce a NaN, which ruins everything. This
  function ensures each probability is at least eps and no more than one
  before taking the log.

  Args:
    y: matrix of true probabilities same size as probs
    probs: matrix of probabilities for the minibatch
    eps: value to clip the probabilities at
    class_weights: vector of relative weights to be assigned to each class
    sumd: dimensions along which to sum the x-ent matrix

  Returns:
    cross entropy loss for each example in the minibatch
  """
#   eps = 0
  adjusted_probs = tf.clip_by_value(probs, eps, 1.0 - eps)
  xent_mat = -y * tf.log(adjusted_probs)
  if class_weights is not None:
    xent_mat *= class_weights

  return xent_mat, adjusted_probs
#   return tf.reduce_sum(xent_mat, sumd)


def _SafeNegEntropy(probs, batch_size, eps=0.0001):
  """Computes negative entropy in a way that will not overflow."""
  adjusted_probs = tf.clip_by_value(probs, eps, 1.0 - eps)
  entropy = tf.multiply(probs, tf.log(adjusted_probs))
  return tf.reduce_sum(entropy) / batch_size


In [0]:
model = WordLevelModel(batch_size=batch_size,
                           max_sequence_len=dataset.max_sequence_len,
                           weights=dataset.w,
                           dropout_keep_prob=0.7,
                           out_vocab_size=len(output_vocab), c2v=c2v)
# model = WordLevelModel(batch_size=batch_size,
#                            max_sequence_len=dataset.max_sequence_len,
#                            dropout_keep_prob=0.7,
#                            out_vocab_size=len(output_vocab), c2v=c2v)
saver = tf.train.Saver(tf.all_variables())
session = tf.Session(config=config)
model.preds_by_word
model.preds_by_instance
model._xent
model.prob_div
model.probs1
# tf.expand_dims(tf.expand_dims(model._mask,1),3)


<tf.Tensor 'wordrnn/Mul:0' shape=(25, 2453, 7) dtype=float32>

In [0]:
scores = []
# confusion_matrix = [] 
def Metrics(preds, labs, show=True):
  """Print precision, recall and F1 for each language.
  Assumes a single language per example, i.e. no code switching.
  Args:
    preds: list of predictions
    labs: list of labels
    show: flag to toggle printing
  """
  all_langs = set(preds + labs)
  preds = np.array(preds)
  labs = np.array(labs)

  label_totals = collections.Counter(labs)
  pred_totals = collections.Counter(preds)
  confusion_matrix = collections.Counter(zip(preds, labs))

  num_correct = 0
  for lang in all_langs:
    num_correct += confusion_matrix[(lang, lang)]
  acc = num_correct / float(len(preds))
  print 'accuracy = {0:.3f}'.format(acc)

  if show:
    print ' Lang     Prec.   Rec.   F1'
    print '------------------------------'

#   scores = []
  fmt_str = '  {0:6}  {1:6.2f} {2:6.2f} {3:6.2f}'
  for lang in sorted(all_langs):
    idx = preds == lang
    total = max(1.0, pred_totals[lang])
    precision = 100.0 * confusion_matrix[(lang, lang)] / total

    idx = labs == lang
    total = max(1.0, label_totals[lang])
    recall = 100.0 * confusion_matrix[(lang, lang)] / total

    if precision + recall == 0.0:
      f1 = 0.0
    else:
      f1 = 2.0 * precision * recall / (precision + recall)

    scores.append([precision, recall, f1])
    if show:
      print fmt_str.format(lang, precision, recall, f1)

  totals = np.array(scores).mean(axis=0)
  if show:
    print '------------------------------'
    print fmt_str.format('Total:', totals[0], totals[1], totals[2])
  return totals[2]


def ConfusionMat(preds, labs):
  """Plot and show a confusion matrix.
  Args:
    preds: list of predicted labels
    labs: list of true labels
  """
  all_langs = set(preds + labs)  # this is the set of all possible labels
  num_langs = len(all_langs)

  # create a mapping from labels to id numbers
  lookup = dict(zip(sorted(all_langs), range(num_langs)))

  # make the counts for the confusion matrix
  counts = np.zeros((num_langs, num_langs))
  for p, l in zip(preds, labs):
    counts[lookup[p], lookup[l]] += 1

  # plot a colormap using log scale
  pyplot.imshow(np.log(counts+1.0), interpolation='none')

  # plot the text labels
  for i in xrange(num_langs):
    for j in xrange(num_langs):
      pyplot.text(i, j, str(int(counts[i, j])), color='white',
                  horizontalalignment='center')
  # take care of the axes
  pyplot.xticks(range(num_langs), sorted(all_langs))
  pyplot.yticks(range(num_langs), sorted(all_langs))
  pyplot.xlabel('Prediction')
  pyplot.ylabel('True Label')
  pyplot.show()
  return 0



In [0]:
def MakeFeedDict(words, seqlen, labs, ws=None):
  """Create the feed dict to process each batch.

  All the inputs should be from the GetNextBatch command.

  Args:
    words: matrix of word ids
    seqlen: vector of sequence lengths
    labs: target matrix
    ws: per-example weights

  Returns:
    dictionary to be used as feed dict.
  """
  batch_data = {
    model.seq_lens: seqlen,
    model.x: words,
  }

#   if mode == 'train':
  batch_data[model.y] = labs
#   batch_data[model.example_weights] = ws
#   batch_data[dropout_keep_prob] = 0.7

#   if not baseline:
  batch_vocab, words_remapped = Char2Vec.GetBatchVocab(words)      ######

  batch_data.update({
    c2v.words_as_chars: the_words[batch_vocab, :],
    model.x: words_remapped
  })

  if hasattr(c2v, 'seq_lens'):
    batch_data.update({
      c2v.seq_lens: word_lengths[batch_vocab],
      c2v.batch_dim: len(batch_vocab)
      })                                                     ################

  return batch_data

In [0]:
###################### train ############################################

dropout_keep_prob = tf.placeholder_with_default(1.0, (), name='keep_prob')

# logging.info('Input Vocab Size: {0}'.format(len(input_vocab)))
# logger.info('Char Vocab Size: {0}'.format(len(char_vocab)))

tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(model.cost, tvars), 5.0)
optimizer = tf.train.AdamOptimizer(0.001)
train_op = optimizer.apply_gradients(zip(grads, tvars))

session.run(tf.initialize_all_variables())
# if args.start:
#   saver.restore(session, os.path.join(args.start, 'model.bin'))
# util.PrintParams(tf.trainable_variables(), handle=logging.info)

# maxitr = model_params.get('num_training_iters', 80001)
# maxitr = 10001
maxitr = 101
print "Training for {} iterations...".format(maxitr)
for idx in xrange(maxitr): 
#   if args.data == 'codeswitch':
#     words, seqlen, labs, ws, lines = dataset.GetNextBatch()
#   else:
#   words = dataset.sentences[[0],:]
#   seqlen = dataset.seq_lens[[0]]
#   labs = dataset.labels[[0],:]
#   ws = dataset.example_weights[[0]]
#   words = dataset.GetNextBatch()
  words, seqlen, labs, ws = dataset.GetNextBatch()
  batch_data = MakeFeedDict(words, seqlen, labs, ws)

#   if idx % 25 == 0:
  print idx
#     probs = session.run([model.probs], batch_data)[0]
#     s = [input_vocab[i] for i in words[0, :seqlen[0]]]
#     print ' '.join(s)
#     print "Predict:", GetTopWordLevelPreds(probs[0,:,:], seqlen[0])
#     print "Actual: ", GetTopWordLevelPreds(labs[0,:,:], seqlen[0])

  cost, inpu, xx, tmp, xnt, ap, pw, pi, p1, pd, p, m, we2, _ = session.run([model.cost, model._inputs, model.x, model.temp, model._xent, model.adjprob, model.preds_by_word, model.preds_by_instance, model.probs1, model.prob_div, model.probs, model._mask, c2v.word_embeddings, train_op], batch_data)
#   cost, _ = session.run([model.cost, train_op], batch_data)
#   preds_by_word, inst = session.run([model.preds_by_word, model.preds_by_instance], batch_data)
  logger.info({'iter': idx, 'cost': float(cost)})
  if idx == maxitr-1 :
    saver.save(session, './model.bin')


In [0]:
# totals

In [0]:
saver.restore(session,  'model.bin')


In [0]:

len(tmp[0][0])      # 1 * 2453 * 210
cost
xnt[0][24]
ap
len(pw[0][0][0])
len(pi[0][0][0])
len(p1[0][0])
len(pd[0])
# len(p[0])
len(ap[0][0])
len(m[0])

ap[0][51]
# p1[0][2]
ap
# len(p)
p1[0][1]

array([0.14389476, 1.0629226 , 0.12079805, 0.0932117 , 0.32992396,
       0.2056115 , 0.04363749], dtype=float32)

In [0]:
################################ eval ###########################################3

# model_vars=[]
# def Eval():
  """Evaluates on dev data.
  Writes results to a results.tsv file in the expdir for use in the 
  scoring script.
  Args:
    expdir: path to experiment directory
  """ 

# saver.restore(session,  'model.bin')

all_preds, all_labs = [], []
pred, lab = [], []
for _ in xrange(test_dataset.GetNumBatches()):
#  words  = dataset.GetNextBatch()
#  words = dataset.sentences[[0],:]
#  seqlen = dataset.seq_lens[[0]]
#  labs = dataset.labels[[0],:]
#  weights = dataset.example_weights[[0]]
 words, seqlen, labs, weights  = test_dataset.GetNextBatch()
 batch_data = MakeFeedDict(words, seqlen, labs, weights)
#  batch_data = MakeFeedDict(words, seqlen, labs)
#  all_words += [words]

 model_vars = [model.probs, model.preds_by_word]
 probs, pp = session.run(model_vars, batch_data)
#  for i in xrange(len(labs)):
#     print labs[i],  probs[i], i
#  prob_mat=np.argmax(probs, axis=2)
#  lab_mat = np.argmax(labs, axis=2)
#  for p in np.argmax(probs[0], axis=1):
#     print str(p) + " . " + output_vocab[p] + " ... "
#  for p in np.argmax(labs[0], axis=1):
#     print str(p) + " - " + output_vocab[p] + " ||| "
 for i in xrange (batch_size):
#     for p in np.argmax(probs[i], axis=1):
#       out_voc = output_vocab[p]
#       if (output_vocab[p]=='und'):
#         out_voca
    for j in xrange(2453):
      p=output_vocab[np.argmax(labs[i][j])]
      if(p=='und'):
        continue
      
      all_preds+=[output_vocab[np.argmax(probs[i][j])]]
      all_labs+=[p]
#     print lab
#     all_preds = pred
#     all_labs = lab
    
  
Metrics(all_preds, all_labs)
# print all_labs
# print all_preds

# This output file is in the format needed to score for TweetLID
ids = test_dataset.GetIds()
with open('results.tsv', 'w') as f:
 for p, l in zip(all_preds, all_labs):
     f.write('{0}\t{1}\n'.format(p,l))
    
totals = np.array(scores).mean(axis=0)
totals


array([11.58665896, 16.66666667, 13.66996474])

In [0]:
# words, seqlen, labs, weights  = dataset.GetNextBatch()
# batch_data = MakeFeedDict(words, seqlen, labs, weights)
# inp, xx = tf.Session().run([model._inputs, model.x], batch_data)

In [0]:
inp = inpu

with open('inputs.tsv', 'w') as f:
 w = 0
 for word in inp[0]:
   for p in inp[0][w]:
       f.write('{0}\t'.format(p))
   f.write('\n')
   w = w+1

with open('we.tsv', 'w') as f:
 for sentence in we2:
   for w in sentence:
       f.write('{0}\t'.format(w))
   f.write('\n')

with open('x.tsv', 'w') as f:
 for sentence in xx[0]:
   f.write('{0}\t'.format(sentence))


In [0]:
input_vocab['i']
# input_vocab['think']
# input_vocab['know']
# input_vocab['it']
# input_vocab['well']
# input_vocab['{']
input_vocab[42]
input_vocab[0]
input_vocab[5]


u'eligible'

In [0]:
c2v.word_embeddings

<tf.Tensor 'c2v/add_1:0' shape=<unknown> dtype=float32>

In [0]:
# inputs - 2453 * 210
# embedd - 46   * 210
len(input_vocab.word_to_idx)

9031